##Carico il corpus wikitext2 (preprocessing già effettuato)

In [2]:
import pandas as pd
import os 
import sys
import nltk
sys.path.append("../")

df = pd.read_csv("../corpus/WikiText2_36718.txt", header=None)
df = df.rename(columns={0: 'token'})
df

wpt = nltk.WordPunctTokenizer()
tokenized = [wpt.tokenize(document) for document in df['token']]


##Training

In [3]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(vector_size=300, sg=1, min_count=50)
w2v_model.build_vocab(tokenized)
w2v_model.train(tokenized, total_examples=w2v_model.corpus_count, epochs=5)


(3457668, 4619235)

##Analysis

In [42]:
similars = w2v_model.wv.most_similar(positive=["rock"])
similars[1][0]


'pop'

##TSNE

In [23]:
# embedding from first model layer

import numpy as np
words = []
keyed = w2v_model.wv
print(len(keyed))
embeddings = []
for val, key in w2v_model.wv.key_to_index.items():
    words.append(val)
    embeddings.append(keyed.get_vector(key))


3520


In [24]:
rock_idx = w2v_model.wv.key_to_index["rock"]
print(rock_idx)

292


In [25]:

print(w2v_model.wv.index_to_key[1])

one


In [36]:
from sklearn.manifold import TSNE

# get embeddings
embeddings_df = pd.DataFrame(embeddings[0:2])

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
embeddings_df_trans.index = words[0:2]
# if token is a number
is_numeric = embeddings_df_trans.index.str.isnumeric()

/home/fmollica/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/fmollica/.local/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [37]:
import numpy as np
import plotly.graph_objs as go

color = np.where(is_numeric, "green", "black")
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=color),
    )
)
fig.show()